In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

import tensorflow as tf
import random
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [2]:
##############
# parameters #
##############
# dontFreezeLast = 0;

# patience = 10;

# gpuName = '/device:GPU:0'
# workers = 2;
# histogram_freq = 0;

# epochs = 100;
# validation_size=0.3;

modelPath = '../models/VGG19_opt/run2.h5';
os.environ["CUDA_VISIBLE_DEVICES"]="0"

First we will read in the csv's so we can see some more information on the filenames and breeds

In [3]:
# df_train = pd.read_csv('../input/labels.csv')
# df_test = pd.read_csv('../input/sample_submission.csv')

# print('Training images: ',df_train.shape[0])
# print('Test images: ',df_test.shape[0])

# reduce dimensionality
#df_train = df_train.head(100)
#df_test = df_test.head(100)

In [4]:
#df_train.head(10)

We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [5]:
# targets_series = pd.Series(df_train['breed'])
# one_hot = pd.get_dummies(targets_series, sparse = True)

In [6]:
# one_hot_labels = np.asarray(one_hot)

Next we will read in all of the images for test and train, using a for loop through the values of the csv files. I have also set an im_size variable which sets the size for the image to be re-sized to, 90x90 px, you should play with this number to see how it affects accuracy.

In [7]:
# im_size = 90

In [8]:
# x_train = []
# y_train = []
# x_test = []

In [9]:
# i = 0 
# for f, breed in tqdm(df_train.values[:10]):
#     img = cv2.imread('../input/train/{}.jpg'.format(f))
#     label = one_hot_labels[i]
#     x_train.append(cv2.resize(img, (im_size, im_size)))
#     y_train.append(label)
#     i += 1

In [10]:
# for f in tqdm(df_test['id'].values):
#     img = cv2.imread('../input/test/{}.jpg'.format(f))
#     x_test.append(cv2.resize(img, (im_size, im_size)))

In [11]:
# y_train_raw = np.array(y_train, np.uint8)
# x_train_raw = np.array(x_train, np.float32) / 255.
# x_test  = np.array(x_test, np.float32) / 255.

We check the shape of the outputs to make sure everyting went as expected.

In [12]:
# print(x_train_raw.shape)
# print(y_train_raw.shape)
# print(x_test.shape)

We can see above that there are 120 different breeds. We can put this in a num_class variable below that can then be used when creating the CNN model.

In [13]:
# num_class = y_train_raw.shape[1]
# print('Number of classes: ', num_class)

It is important to create a validation set so that you can gauge the performance of your model on independent data, unseen to the model in training. We do this by splitting the current training set (x_train_raw) and the corresponding labels (y_train_raw) so that we set aside 30 % of the data at random and put these in validation sets (X_valid and Y_valid).

* This split needs to be improved so that it contains images from every class, with 120 separate classes some can not be represented and so the validation score is not informative.

In [14]:
# X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=validation_size, random_state=1)

Now we build the CNN architecture. Here we are using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview). Unfortunately it doesn't seem possible to downlod the weights from within this kernel so make sure you set the weights argument to 'imagenet' and not None, as it currently is below.

We then remove the final layer and instead replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [15]:
def data():
    print('Getting data')
    df_train = pd.read_csv('../input/labels.csv')
    df_test = pd.read_csv('../input/sample_submission.csv')
    
    targets_series = pd.Series(df_train['breed'])
    one_hot = pd.get_dummies(targets_series, sparse = True)
    one_hot_labels = np.asarray(one_hot)
    
    im_size = 90
    x_train = []
    y_train = []
    x_test = []
    
    i = 0 
    for f, breed in tqdm(df_train.values):
        img = cv2.imread('../input/train/{}.jpg'.format(f))
        label = one_hot_labels[i]
        x_train.append(cv2.resize(img, (im_size, im_size)))
        y_train.append(label)
        i += 1
    
    y_train_raw = np.array(y_train, np.uint8)
    x_train_raw = np.array(x_train, np.float32) / 255.
    num_class = y_train_raw.shape[1]
    
    print('Splitting into training/validation')
    X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)
    
    return X_train, Y_train, X_valid, Y_valid

In [ ]:
# prepare data and model for hyperas

def model(X_train,Y_train,X_valid,Y_valid):
    print('Creating model')
    base_model = VGG19(weights = 'imagenet',
                       include_top=False,
                       input_shape=(im_size, im_size, 3))

    dropout = {{uniform(0.5,1)}};
    layers = {{choice([0,1,2])}};
    dontFreeze = {{choice(list(range(5+1)))}};
    batchSize = {{choice([16,64,256])}};
    
    print()
    print('dropout=',dropout)
    print('layers=',layers)
    print('dontFreeze=',dontFreeze)
    print('batchSize=',batchSize)
    print()
    
    stepsPerEpoch = round( len(X_train)/batchSize );
    
    # Add a new top layer
    x = base_model.output
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    if layers>=2:
        x = Dense(1024,activation='relu')(x)
    if layers>=1:
        x = Dense(512,activation='relu')(x)
    # in any case:
    predictions = Dense(num_class, activation='softmax')(x)

    # This is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # First: train only the top layers (which were randomly initialized)
    for i in range(len(base_model.layers)-dontFreeze):
        base_model.layers[i].trainable = False

    # predetermined optimizer
    lr=0.00020389590556056983;
    beta_1=0.9453158868247398;
    beta_2=0.9925872692991417;
    decay=0.000821336141287018;
    adam = keras.optimizers.Adam(lr=lr,beta_1=beta_1,beta_2=beta_2,decay=decay)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=adam, 
                  metrics=['accuracy'])

    callbacks_list = [];
    callbacks_list.append(keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=10,
        verbose=1));


    # data augmentation & fitting
    datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.5,
        zoom_range=0.5,
        horizontal_flip=True,
        vertical_flip=True);
    
    model.fit_generator(
        datagen.flow(X_train,Y_train,batch_size=batchSize),
        steps_per_epoch=stepsPerEpoch,
        epochs=150,
        verbose=1,
        validation_data=(X_valid,Y_valid),
        workers=2,
        shuffle=True,
        callbacks=callbacks_list)
#     model.fit(X_train, Y_train,
#       epochs=100,
#       batch_size = batchSize,
#       validation_data=(X_valid, Y_valid),
#       verbose=1,
#       callbacks=callbacks_list)

    score, acc = model.evaluate(X_valid, Y_valid, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}


In [ ]:
best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='VGG19Optimization_2')

X_train, Y_train, X_test, Y_test = data()
val_loss, val_acc = best_model.evaluate(X_test, Y_test);
print("Evalutation of best performing model:")
print("Validation loss: ", val_loss)
print("Validation accuracy: ", val_acc)

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.applications.vgg19 import VGG19
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.layers import Dense, Dropout, Flatten
except:
    pass

try:
    from keras.preprocessing.image import ImageDataGenerator
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import random
except:
    pass

try:
    import os
except:
    pass

try:
    from tqdm import tqdm
except:
    pass

try:
    from sklearn import preprocessing
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    import cv2
except:
   

100%|██████████| 10222/10222 [01:07<00:00, 151.07it/s]


Splitting into training/validation
Creating model

dropout= 0.6064002165637792
layers= 2
dontFreeze= 3
batchSize= 64

Epoch 1/150
112/112 [==============================] - 36s 318ms/step - loss: 4.8143 - acc: 0.0085 - val_loss: 4.7743 - val_acc: 0.0170
Epoch 2/150
112/112 [==============================] - 31s 275ms/step - loss: 4.7729 - acc: 0.0114 - val_loss: 4.7211 - val_acc: 0.0192
Epoch 3/150
112/112 [==============================] - 31s 277ms/step - loss: 4.7246 - acc: 0.0151 - val_loss: 4.6869 - val_acc: 0.0199
Epoch 4/150
112/112 [==============================] - 29s 261ms/step - loss: 4.7050 - acc: 0.0163 - val_loss: 4.6554 - val_acc: 0.0274
Epoch 5/150
112/112 [==============================] - 29s 263ms/step - loss: 4.6877 - acc: 0.0212 - val_loss: 4.6124 - val_acc: 0.0323
Epoch 6/150
112/112 [==============================] - 29s 256ms/step - loss: 4.6618 - acc: 0.0215 - val_loss: 4.5934 - val_acc: 0.0287
Epoch 7/150
112/112 [==============================] - 29s 260ms/s

112/112 [==============================] - 30s 267ms/step - loss: 3.6300 - acc: 0.1475 - val_loss: 3.5773 - val_acc: 0.1647
Epoch 61/150
112/112 [==============================] - 29s 261ms/step - loss: 3.6129 - acc: 0.1509 - val_loss: 3.5505 - val_acc: 0.1686
Epoch 62/150
112/112 [==============================] - 29s 258ms/step - loss: 3.6383 - acc: 0.1443 - val_loss: 3.5262 - val_acc: 0.1699
Epoch 63/150
112/112 [==============================] - 29s 257ms/step - loss: 3.6061 - acc: 0.1547 - val_loss: 3.5158 - val_acc: 0.1682
Epoch 64/150
112/112 [==============================] - 29s 259ms/step - loss: 3.6075 - acc: 0.1488 - val_loss: 3.5224 - val_acc: 0.1673
Epoch 65/150
112/112 [==============================] - 29s 258ms/step - loss: 3.5862 - acc: 0.1541 - val_loss: 3.5191 - val_acc: 0.1679
Epoch 66/150
112/112 [==============================] - 30s 267ms/step - loss: 3.5619 - acc: 0.1568 - val_loss: 3.5017 - val_acc: 0.1728
Epoch 67/150
112/112 [==============================] 

28/28 [==============================] - 23s 832ms/step - loss: 4.9210 - acc: 0.0098 - val_loss: 4.7671 - val_acc: 0.0186
Epoch 5/150
28/28 [==============================] - 23s 830ms/step - loss: 4.8821 - acc: 0.0121 - val_loss: 4.7640 - val_acc: 0.0196
Epoch 6/150
28/28 [==============================] - 23s 831ms/step - loss: 4.8649 - acc: 0.0129 - val_loss: 4.7598 - val_acc: 0.0192
Epoch 7/150
28/28 [==============================] - 23s 830ms/step - loss: 4.8382 - acc: 0.0124 - val_loss: 4.7571 - val_acc: 0.0225
Epoch 8/150
28/28 [==============================] - 23s 830ms/step - loss: 4.8354 - acc: 0.0138 - val_loss: 4.7552 - val_acc: 0.0215
Epoch 9/150
28/28 [==============================] - 23s 832ms/step - loss: 4.8353 - acc: 0.0135 - val_loss: 4.7544 - val_acc: 0.0225
Epoch 10/150
28/28 [==============================] - 23s 830ms/step - loss: 4.8227 - acc: 0.0148 - val_loss: 4.7531 - val_acc: 0.0215
Epoch 11/150
28/28 [==============================] - 23s 830ms/step - lo

...

Epoch 48/150
28/28 [==============================] - 24s 842ms/step - loss: 3.8482 - acc: 0.1287 - val_loss: 3.7008 - val_acc: 0.1559
Epoch 49/150
28/28 [==============================] - 24s 843ms/step - loss: 3.8270 - acc: 0.1280 - val_loss: 3.6909 - val_acc: 0.1594
Epoch 50/150
28/28 [==============================] - 24s 843ms/step - loss: 3.8108 - acc: 0.1301 - val_loss: 3.6794 - val_acc: 0.1562
Epoch 51/150
28/28 [==============================] - 24s 843ms/step - loss: 3.8330 - acc: 0.1265 - val_loss: 3.6855 - val_acc: 0.1572
Epoch 52/150
28/28 [==============================] - 24s 843ms/step - loss: 3.8166 - acc: 0.1346 - val_loss: 3.6772 - val_acc: 0.1585
Epoch 53/150
28/28 [==============================] - 24s 842ms/step - loss: 3.8104 - acc: 0.1352 - val_loss: 3.6703 - val_acc: 0.1572
Epoch 54/150
28/28 [==============================] - 24s 845ms/step - loss: 3.8106 - acc: 0.1288 - val_loss: 3.6763 - val_acc: 0.1607
Epoch 55/150
28/28 [==============================] - 2

112/112 [==============================] - 28s 251ms/step - loss: 4.4658 - acc: 0.0420 - val_loss: 4.3560 - val_acc: 0.0574
Epoch 24/150
112/112 [==============================] - 28s 251ms/step - loss: 4.4611 - acc: 0.0447 - val_loss: 4.3238 - val_acc: 0.0623
Epoch 25/150
112/112 [==============================] - 28s 251ms/step - loss: 4.4369 - acc: 0.0409 - val_loss: 4.2971 - val_acc: 0.0580
Epoch 26/150
112/112 [==============================] - 28s 252ms/step - loss: 4.4300 - acc: 0.0463 - val_loss: 4.2753 - val_acc: 0.0681
Epoch 27/150
112/112 [==============================] - 28s 251ms/step - loss: 4.4308 - acc: 0.0428 - val_loss: 4.2666 - val_acc: 0.0629
Epoch 28/150
112/112 [==============================] - 28s 251ms/step - loss: 4.4167 - acc: 0.0488 - val_loss: 4.2365 - val_acc: 0.0675
Epoch 29/150
112/112 [==============================] - 28s 252ms/step - loss: 4.3970 - acc: 0.0443 - val_loss: 4.2336 - val_acc: 0.0665
Epoch 30/150
112/112 [==============================] 

/home/mattia-lecci/venv_project/lib/python3.5/site-packages/keras/callbacks.py:116: UserWarning: Method on_batch_end() is slow compared to the batch update (0.237195). Check your callbacks.
  % delta_t_median)


112/112 [==============================] - 29s 261ms/step - loss: 4.1830 - acc: 0.0629 - val_loss: 3.9523 - val_acc: 0.0981
Epoch 45/150
112/112 [==============================] - 28s 251ms/step - loss: 4.1536 - acc: 0.0689 - val_loss: 3.9722 - val_acc: 0.0936
Epoch 46/150
112/112 [==============================] - 28s 252ms/step - loss: 4.1401 - acc: 0.0725 - val_loss: 3.9343 - val_acc: 0.1034
Epoch 47/150
112/112 [==============================] - 28s 251ms/step - loss: 4.1246 - acc: 0.0774 - val_loss: 3.8975 - val_acc: 0.1060
Epoch 48/150
112/112 [==============================] - 28s 251ms/step - loss: 4.1012 - acc: 0.0794 - val_loss: 3.8756 - val_acc: 0.1109
Epoch 49/150
112/112 [==============================] - 28s 252ms/step - loss: 4.0903 - acc: 0.0791 - val_loss: 3.8873 - val_acc: 0.1092
Epoch 50/150
112/112 [==============================] - 28s 253ms/step - loss: 4.0853 - acc: 0.0800 - val_loss: 3.8774 - val_acc: 0.1151
Epoch 51/150
112/112 [==============================] 

 21%|██        | 2164/10222 [01:06<04:09, 32.31it/s] 

In [ ]:
best_model.save(modelPath);
best_model.summary()

Remember, accuracy is low here because we are not taking advantage of the pre-trained weights as they cannot be downloaded in the kernel. This means we are training the wights from scratch and I we have only run 1 epoch due to the hardware constraints in the kernel.

Next we will make our predictions.

In [ ]:
# preds = model.predict(x_test, verbose=1)

In [ ]:
# sub = pd.DataFrame(preds)
# # Set column names to those generated by the one-hot encoding earlier
# col_names = one_hot.columns.values
# sub.columns = col_names
# # Insert the column id from the sample_submission at the start of the data frame
# sub.insert(0, 'id', df_test['id'])
# sub.head(10)